#### Cleaning data and Seperating into the Different Frames

In [26]:
# panda
import pandas as pd
import os

In [27]:
os.chdir('C:/Users/dalto/OneDrive/Pictures/Documents/Projects/MLB Season Model/') # wd

In [2]:
fg = pd.read_csv('fg_00_24.csv')
stc = pd.read_csv('stc_15_24.csv')

find the data cutoffs based on year to help decide data seperation

In [3]:
def find_latest_missing_year(df, year_col_name):
    latest_missing_year = {}
    missing_counts = {}
    for col in df.columns:
        missing_data = df[df[col].isnull()]
        missing_counts[col] = df[col].isnull().sum()
        if not missing_data.empty:
            latest_year = missing_data[year_col_name].max()
            latest_missing_year[col] = latest_year
        else:
            latest_missing_year[col] = None  # No missing values in this column
    return pd.Series(latest_missing_year), pd.Series(missing_counts)

fg_missing_years, fg_missing_counts = find_latest_missing_year(fg, 'Season')
stc_missing_years, stc_missing_counts = find_latest_missing_year(stc, 'year')

print("FG Latest Missing Years:\n", fg_missing_years)
print("\nFG Missing Value Counts:\n", fg_missing_counts)
print("\nSTC Latest Missing Years:\n", stc_missing_years)
print("\nSTC Missing Value Counts:\n", stc_missing_counts)

FG Latest Missing Years:
 Season           NaN
Name             NaN
Team             NaN
PA               NaN
Age              NaN
BB%              NaN
K%               NaN
BABIP            NaN
wOBA             NaN
xwOBA         2024.0
wRC+             NaN
BsR              NaN
Off              NaN
Def              NaN
WAR              NaN
Barrel%       2014.0
maxEV         2014.0
HardHit%      2014.0
xSLG          2024.0
BsR.1            NaN
O-Swing%      2001.0
Z-Swing%      2001.0
O-Contact%    2007.0
Z-Contact%    2001.0
Contact%      2001.0
SwStr%        2001.0
CSW%          2001.0
NameASCII        NaN
PlayerId         NaN
MLBAMID          NaN
dtype: float64

FG Missing Value Counts:
 Season            0
Name              0
Team              0
PA                0
Age               0
BB%               0
K%                0
BABIP             0
wOBA              0
xwOBA         14863
wRC+              0
BsR               0
Off               0
Def               0
WAR               0
Ba

basic cleaning and merge

In [4]:
fg.drop(columns=['xwOBA', 'wOBA', 'NameASCII', 'PlayerId', 'MLBAMID', 'xSLG', 'BsR.1'], inplace=True)

In [5]:
# fix name cols so able to merge
fg['Name'] = fg['Name'].str.lower()
stc = stc.rename(columns={'last_name, first_name': 'Name', 'year': 'Season'}) # rename cols so easier
stc['Name'] = stc['Name'].str.lower()
stc['Name'] = stc['Name'].str.split(', ').str[::-1].str.join(' ')
stc.head()

,Name,player_id,Season,pa,k_percent,bb_percent,woba,xwoba,sweet_spot_percent,barrel_batted_rate,hard_hit_percent,avg_best_speed,avg_hyper_speed,whiff_percent,swing_percent
0,bartolo colon,112526,2015,64,37.5,0.0,0.136,0.153,23.1,0.0,15.4,93.759829,91.247927,30.6,58.1
1,torii hunter,116338,2015,567,18.5,6.2,0.304,0.290,28.5,5.0,34.9,98.563404,93.393480,23.1,53.4
2,david ortiz,120074,2015,614,15.5,12.5,0.379,0.420,34.8,13.1,49.1,102.851133,96.053058,23.2,44.7
3,alex rodriguez,121347,2015,620,23.4,13.5,0.361,0.368,31.4,10.9,43.9,101.381141,95.014380,32.0,43.9
4,aramis ramirez,133380,2015,516,13.2,6.0,0.309,0.304,33.5,5.6,34.5,97.851256,92.944763,17.9,52.9


remove special chars for easy matching

In [6]:
import unicodedata
import re

def clean_name(name):
    # Normalize the string to decompose accented characters
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    # Remove all non-alphanumeric characters
    name = re.sub(r'[^a-z\s]', '', name)
    return name

fg['Name'] = fg['Name'].apply(clean_name)
stc['Name'] = stc['Name'].apply(clean_name)

#### create data sets

##### 2015

In [7]:
fg15 = fg[fg['Season'] >= 2015]
full_15 = fg15.merge(stc, how='outer', on=['Name', 'Season'])

clean to account for pitchers

In [11]:
full_15 = full_15[full_15['Team'].notna()]
full_15.head()

,Season,Name,Team,PA,Age,BB%,K%,BABIP,wRC+,BsR,...,bb_percent,woba,xwoba,sweet_spot_percent,barrel_batted_rate,hard_hit_percent,avg_best_speed,avg_hyper_speed,whiff_percent,swing_percent
0,2015,aaron altherr,PHI,161.0,24.0,0.099379,0.254658,0.301075,124.351495,1.049130,...,9.9,0.353,0.312,33.3,3.0,43.4,100.953732,94.773502,25.5,41.2
1,2016,aaron altherr,PHI,227.0,25.0,0.101322,0.303965,0.288000,63.946339,-0.271549,...,10.1,0.266,0.282,28.7,3.1,34.9,99.056324,93.591569,28.9,43.3
2,2017,aaron altherr,PHI,412.0,26.0,0.077670,0.252427,0.328000,119.718295,-1.130484,...,7.8,0.359,0.341,32.7,10.4,42.8,102.082151,95.188570,29.9,44.4
3,2018,aaron altherr,PHI,285.0,27.0,0.126316,0.319298,0.246575,73.992787,0.623974,...,12.6,0.282,0.317,30.5,9.7,45.8,102.305923,95.402841,32.8,44.8
4,2019,aaron altherr,- - -,66.0,28.0,0.045455,0.378788,0.111111,-22.663759,0.341888,...,4.5,0.135,0.210,32.4,5.4,21.6,95.495542,91.988153,35.0,51.1


##### 2023

In [9]:
fg23 = fg[fg['Season'] >= 2023]
full_23 = fg23.merge(stc, how='outer', on=['Name', 'Season'])

In [12]:
full_23 = full_23[full_23['Team'].notna()]
full_23.head()

,Season,Name,Team,PA,Age,BB%,K%,BABIP,wRC+,BsR,...,bb_percent,woba,xwoba,sweet_spot_percent,barrel_batted_rate,hard_hit_percent,avg_best_speed,avg_hyper_speed,whiff_percent,swing_percent
15,2023,aaron hicks,- - -,312.0,33.0,0.134615,0.221154,0.310881,108.904972,1.063874,...,13.5,0.326,0.282,26.4,4.0,29.4,96.660791,92.357553,25.6,40.3
16,2024,aaron hicks,LAA,63.0,34.0,0.095238,0.365079,0.212121,19.982590,0.163153,...,9.5,0.196,0.248,32.4,5.9,35.3,99.638574,94.126470,32.8,44.0
27,2023,aaron judge,NYY,458.0,31.0,0.192140,0.283843,0.300493,172.114957,-2.401622,...,19.2,0.420,0.461,37.1,27.5,64.2,107.408656,99.748784,36.6,41.3
28,2024,aaron judge,NYY,704.0,32.0,0.188920,0.242898,0.367470,218.425380,-0.446974,...,18.9,0.476,0.479,40.8,26.9,61.0,107.200436,99.103702,30.7,42.0
38,2024,aaron schunk,COL,98.0,26.0,0.040816,0.316327,0.327869,49.470988,-0.647032,...,4.1,0.257,0.236,39.7,4.8,31.7,95.868962,91.961485,27.6,58.5


2002

In [25]:
full_02 = fg[fg['Season'] > 2001]
full_02 = full_02.drop(columns=['Barrel%', 'maxEV', 'HardHit%'])
full_02['O-Contact%'] = full_02['O-Contact%'].fillna(full_02.mean(numeric_only=True)['O-Contact%'])

#### Export all the data

In [28]:
full_02.to_csv('./data/cleaned/data_02_24.csv')
full_15.to_csv('./data/cleaned/data_15_24.csv')
full_23.to_csv('./data/cleaned/data_23_24.csv')